In [1]:
%run DQN_NET.py
# %run PPO_NET.py

Using device: cuda


/mnt/workspace/DQN_NET.py:108: RuntimeWarning: invalid value encountered in scalar divide
  partition_UAV_normalized = (partition_UAV - np.min(partition_UAV)) / (np.max(partition_UAV) - np.min(partition_UAV))
/mnt/workspace/DQN_NET.py:116: RuntimeWarning: invalid value encountered in scalar divide
  partition_car_normalized = (partition_car - np.min(partition_car)) / (np.max(partition_car) - np.min(partition_car))


episode 0 UAV action is1,car action is 4,car reward is -0.2217401566679699
episode 10 UAV action is4,car action is 1,car reward is -0.22469910944055943
episode 20 UAV action is1,car action is 1,car reward is -0.22469910944055943
episode 30 UAV action is1,car action is 2,car reward is -0.23108723524009325
episode 40 UAV action is5,car action is 6,car reward is -0.11402850075757574
episode 50 UAV action is4,car action is 3,car reward is -0.23513653216783215
episode 60 UAV action is6,car action is 1,car reward is -0.22469910944055943
episode 70 UAV action is4,car action is 5,car reward is -0.08652560594405594
episode 80 UAV action is1,car action is 5,car reward is -0.08652560594405594
episode 90 UAV action is4,car action is 6,car reward is -0.11402850075757574
episode 100 UAV action is3,car action is 6,car reward is -0.11402850075757574
episode 110 UAV action is4,car action is 5,car reward is -0.08652560594405594
episode 120 UAV action is4,car action is 4,car reward is -0.2217401566679699

In [7]:
for item in history_1:
    print(item)

-0.2217401566679699
-0.22469910944055943
-0.22469910944055943
-0.23108723524009325
-0.11402850075757574
-0.23513653216783215
-0.22469910944055943
-0.08652560594405594
-0.08652560594405594
-0.11402850075757574
-0.11402850075757574
-0.08652560594405594
-0.2217401566679699
-0.08652560594405594
-0.2217401566679699
-0.08652560594405594
-0.08652560594405594
-0.22469910944055943
-0.11402850075757574
-0.11402850075757574
-0.33035584912587407
-0.08652560594405594
-0.11402850075757574
-0.08652560594405594
-0.08652560594405594
-0.23108723524009325
-0.23513653216783215
-0.11402850075757574
-0.08652560594405594
-0.08652560594405594
-0.33035584912587407
-0.11402850075757574
-0.11402850075757574
-0.08652560594405594
-0.11402850075757574
-0.08652560594405594
-0.08652560594405594
-0.11402850075757574
-0.11402850075757574
-0.11402850075757574
-0.11402850075757574
-0.08652560594405594
-0.08652560594405594
-0.11402850075757574
-0.11402850075757574
-0.08652560594405594
-0.08652560594405594
-0.1140285007575

In [3]:
1
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.distributions import Normal, Categorical


# 检查GPU可用性
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 已有数据（保持不变）
seed_value = 1234
np.random.seed(seed_value)
UNIT = 40
MAZE_H = 8
MAZE_W = 8
utilization_ratios_device = 0.1
utilization_ratios_server = 0.1
d_flop = 1.3 * 10**12
d_power = 20
car_flop = 1.3 * 10**12*0.5
car_power = 20
e_flop = 330 * 10**12
e_power = 450
d_fai = 5*10**-29
trans_v_up = 100*1024*1024/4 #553081138.4484484
trans_v_dn = 20*1024*1024/4
p_cm = 0.1
num_users = 5
# nums_data = np.array([5, 3, 4, 7, 9])  # 客户端本地数据量
partition_point = [1, 2, 3, 4, 5, 6, 7]

device_load = [0.3468e9, 0.3519e9, 2.3408e9, 2.3409e9, 5.3791e9, 9.6951e9, 12.077e9]
server_load = [11.7321e9, 11.727e9, 9.7381e9, 9.738e9, 6.6998e9, 2.3838e9, 0.0019e9]
exchanged_data = [2359296, 2359296, 2359296, 2359296, 1179628, 589824, 294912]
privacy_leak = [0.96122, 0.608901, 0.57954889, 0.593044, 0.535525, 0.007155, 0.054303]
# 转换为NumPy数组
np_partition = np.array(partition_point)
np_device = np.array(device_load)
np_server = np.array(server_load)
np_exchanged = np.array(exchanged_data)
np_privacy = np.array(privacy_leak)


Using device: cuda


In [4]:
def cost_cal(num_data, v_flop, device_power, partition_index):
    partial_device = np_device[partition_index]
    device_time = partial_device * num_data / (v_flop *utilization_ratios_device)

    partial_server = np_server[partition_index]
    server_time = partial_server * num_data / ( e_flop* utilization_ratios_server + 1e-8)
    # print(f"device_time is {device_time}, server_time is {server_time}, cal_time is {device_time+server_time}")

    feature = np_exchanged[partition_index]
    trans_t_up = feature / trans_v_up * num_data
    # print(f"device_time is {device_time}, server_time is {server_time}, cal_time is {device_time+server_time},trans_t_up is {trans_t_up}")
    energy_cal = ((partial_device * device_power) / v_flop + (
            partial_server * e_power * utilization_ratios_server) / e_flop) * num_data
    energy_trans = num_data * p_cm * trans_t_up
    energy = energy_cal + energy_trans
    # print(f"energy cal is{energy_cal}, trans is {energy_trans}")
    landa_trans = 1
    time_all = device_time + server_time + landa_trans * trans_t_up
    return time_all, energy

for partition_num in range(len(partition_point)):
    time_UAV, energy_UAV = cost_cal(3, e_power, partition_num)
    time_car, energy_car = cost_cal(1, e_power, partition_num)
            # total_time += time
    reward = -(time_car+time_UAV)*0.4 - (energy_car + energy_UAV)*0.3 - 0.3*np_privacy[partition_num]
    print(f"partition is {partition_num+1}:time is {time_car+time_UAV}, energy is {energy_car + energy_UAV}, reward is {reward}")

TypeError: cost_cal() missing 1 required positional argument: 'partition_index'

In [ ]:
partition is 1:time is 0.37209284195804193, energy is 0.11774086573426576, reward is -0.1841593965034965
partition is 2:time is 0.37224914685314686, energy is 0.11805193006993009, reward is -0.18431523776223777
partition is 3:time is 0.43320499114219113, energy is 0.23936092167832168, reward is -0.24509027296037297
partition is 4:time is 0.4332080559440559, energy is 0.23936702097902096, reward is -0.24509332867132866
partition is 5:time is 0.3463198144426537, energy is 0.37967521188572173, reward is -0.252430489342778
partition is 6:time is 0.3885997146853147, energy is 0.6204217930069931, reward is -0.3415664237762238
partition is 7:time is 0.4166002303030303, energy is 0.7544510363636363, reward is -0.392975403030303

partition is 1:time is 1.9258080894039906, energy is 0.49161794077949655, reward is -1.397227456660743
partition is 2:time is 2.2242695102897714, energy is 1.0885404886116645, reward is -1.6658427060640064
partition is 3:time is 1.287499347292466, energy is 0.8950001397391748, reward is -0.9907495570786437
partition is 4:time is 1.5761146182108812, energy is 1.4722303973335804, reward is -1.2505032724809748
partition is 5:time is 1.1193444552135756, energy is 1.3986900484610911, reward is -0.923410123495612
partition is 6:time is 1.9704210430177713, energy is 3.100842385887664, reward is -1.6893789687012064
partition is 7:time is 1.7536508800204662, energy is 3.0873020370151742, reward is -1.5362858197158435
partition is 8:time is 2.0348815384633565, energy is 3.6497630769312583, reward is -1.7893933846174752

device_time is 8.892307692307693e-07, server_time is 7.90040404040404e-10, cal_time is 8.900208096348096e-07
device_time is 9.023076923076923e-07, server_time is 7.896969696969697e-10, cal_time is 9.030973892773892e-07
device_time is 6.002051282051282e-06, server_time is 6.557643097643098e-10, cal_time is 6.002707046361046e-06
device_time is 6.002307692307692e-06, server_time is 6.557575757575757e-10, cal_time is 6.002963449883449e-06
device_time is 1.3792564102564103e-05, server_time is 4.5116498316498315e-10, cal_time is 1.3793015267547268e-05
device_time is 2.4859230769230768e-05, server_time is 1.6052525252525252e-10, cal_time is 2.4859391294483293e-05
device_time is 3.096666666666667e-05, server_time is 1.2794612794612793e-13, cal_time is 3.09666667946128e-05

trans_t_up is 0.45
trans_t_up is 0.45
trans_t_up is 0.45
trans_t_up is 0.45
trans_t_up is 0.22499618530273438
trans_t_up is 0.1125
trans_t_up is 0.05625